# Synthetic Data Generation Using RAGAS - RAG Evaluation with LangSmith

In the following notebook we'll explore a use-case for RAGAS' synthetic testset generation workflow!



- 🤝 BREAKOUT ROOM #1
  1. Use RAGAS to Generate Synthetic Data

- 🤝 BREAKOUT ROOM #2
  1. Load them into a LangSmith Dataset
  2. Evaluate our RAG chain against the synthetic test data
  3. Make changes to our pipeline
  4. Evaluate the modified pipeline

SDG is a critical piece of the puzzle, especially for early iteration! Without it, it would not be nearly as easy to get high quality early signal for our application's performance.

Let's dive in!

# 🤝 BREAKOUT ROOM #1

## Task 1: Dependencies and API Keys

We'll need to install a number of API keys and dependencies, since we'll be leveraging a number of great technologies for this pipeline!

1. OpenAI's endpoints to handle the Synthetic Data Generation
2. OpenAI's Endpoints for our RAG pipeline and LangSmith evaluation
3. QDrant as our vectorstore
4. LangSmith for our evaluation coordinator!

Let's install and provide all the required information below!

## Dependencies and API Keys:

> **NOTE: DO NOT RUN THESE CELLS IF YOU ARE RUNNING THIS NOTEBOOK LOCALLY**

In [ ]:
#!pip install -qU ragas==0.2.10

In [ ]:
#!pip install -qU langchain-community==0.3.14 langchain-openai==0.2.14 unstructured==0.16.12 langgraph==0.2.61 langchain-qdrant==0.2.0

### NLTK Import

To prevent errors that may occur based on OS - we'll import NLTK and download the needed packages to ensure correct handling of data.

In [2]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /Users/family/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/family/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [3]:
import os
import getpass

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass("LangChain API Key:")

We'll also want to set a project name to make things easier for ourselves.

In [4]:
from uuid import uuid4

os.environ["LANGCHAIN_PROJECT"] = f"AIM - SDG - {uuid4().hex[0:8]}"

OpenAI's API Key!

In [5]:
os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

## Generating Synthetic Test Data

We wil be using Ragas to build out a set of synthetic test questions, references, and reference contexts. This is useful because it will allow us to find out how our system is performing.

> NOTE: Ragas is best suited for finding *directional* changes in your LLM-based systems. The absolute scores aren't comparable in a vacuum.

### Data Preparation

We'll prepare our data - which should hopefull be familiar at this point since it's our Loan Data use-case!

Next, let's load our data into a familiar LangChain format using the `DirectoryLoader`.

In [6]:
from langchain_community.document_loaders import DirectoryLoader
from langchain_community.document_loaders import PyMuPDFLoader


path = "data/"
loader = DirectoryLoader(path, glob="*.pdf", loader_cls=PyMuPDFLoader)
docs = loader.load()

### Knowledge Graph Based Synthetic Generation

Ragas uses a knowledge graph based approach to create data. This is extremely useful as it allows us to create complex queries rather simply. The additional testset complexity allows us to evaluate larger problems more effectively, as systems tend to be very strong on simple evaluation tasks.

Let's start by defining our `generator_llm` (which will generate our questions, summaries, and more), and our `generator_embeddings` which will be useful in building our graph.

#### 🎬 Study Note
Using 'cheap' model for generator, because it is good enough

### Unrolled SDG

In [7]:
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
generator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4.1-nano"))
generator_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings())

Next, we're going to instantiate our Knowledge Graph.

This graph will contain N number of nodes that have M number of relationships. These nodes and relationships (AKA "edges") will define our knowledge graph and be used later to construct relevant questions and responses.

In [8]:
from ragas.testset.graph import KnowledgeGraph

kg = KnowledgeGraph()
kg

KnowledgeGraph(nodes: 0, relationships: 0)

The first step we're going to take is to simply insert each of our full documents into the graph. This will provide a base that we can apply transformations to.

In [9]:
from ragas.testset.graph import Node, NodeType

### NOTICE: We're using a subset of the data for this example - this is to keep costs/time down.
for doc in docs[:20]:
    kg.nodes.append(
        Node(
            type=NodeType.DOCUMENT,
            properties={"page_content": doc.page_content, "document_metadata": doc.metadata}
        )
    )
kg

KnowledgeGraph(nodes: 20, relationships: 0)

Now, we'll apply the *default* transformations to our knowledge graph. This will take the nodes currently on the graph and transform them based on a set of [default transformations](https://docs.ragas.io/en/latest/references/transforms/#ragas.testset.transforms.default_transforms).

These default transformations are dependent on the corpus length, in our case:

- Producing Summaries -> produces summaries of the documents
- Extracting Headlines -> finding the overall headline for the document
- Theme Extractor -> extracts broad themes about the documents

It then uses cosine-similarity and heuristics between the embeddings of the above transformations to construct relationships between the nodes.

#### 🎬 Study Note:
Summaries, Headlines, and Theme - are Ragas defaults; not something we specify/specified

In [10]:
from ragas.testset.transforms import default_transforms, apply_transforms

transformer_llm = generator_llm
embedding_model = generator_embeddings

default_transforms = default_transforms(documents=docs, llm=transformer_llm, embedding_model=embedding_model)
apply_transforms(kg, default_transforms)
kg

Applying HeadlinesExtractor:   0%|          | 0/17 [00:00<?, ?it/s]

Applying HeadlineSplitter:   0%|          | 0/20 [00:00<?, ?it/s]

unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node


Applying SummaryExtractor:   0%|          | 0/31 [00:00<?, ?it/s]

Property 'summary' already exists in node '163ae0'. Skipping!
Property 'summary' already exists in node '5730d7'. Skipping!
Property 'summary' already exists in node 'f4a2af'. Skipping!
Property 'summary' already exists in node 'a594f3'. Skipping!
Property 'summary' already exists in node '5dcd6f'. Skipping!
Property 'summary' already exists in node '0afffc'. Skipping!
Property 'summary' already exists in node '0418ce'. Skipping!
Property 'summary' already exists in node 'cc5108'. Skipping!
Property 'summary' already exists in node '03d8d1'. Skipping!
Property 'summary' already exists in node '767f4b'. Skipping!
Property 'summary' already exists in node '519ec9'. Skipping!
Property 'summary' already exists in node 'd3ec53'. Skipping!
Property 'summary' already exists in node 'ed714b'. Skipping!
Property 'summary' already exists in node 'fd946f'. Skipping!


Applying CustomNodeFilter:   0%|          | 0/6 [00:00<?, ?it/s]

Applying [EmbeddingExtractor, ThemesExtractor, NERExtractor]:   0%|          | 0/43 [00:00<?, ?it/s]

Property 'summary_embedding' already exists in node '163ae0'. Skipping!
Property 'summary_embedding' already exists in node 'f4a2af'. Skipping!
Property 'summary_embedding' already exists in node '5dcd6f'. Skipping!
Property 'summary_embedding' already exists in node '5730d7'. Skipping!
Property 'summary_embedding' already exists in node '767f4b'. Skipping!
Property 'summary_embedding' already exists in node 'a594f3'. Skipping!
Property 'summary_embedding' already exists in node '0afffc'. Skipping!
Property 'summary_embedding' already exists in node 'd3ec53'. Skipping!
Property 'summary_embedding' already exists in node '0418ce'. Skipping!
Property 'summary_embedding' already exists in node '519ec9'. Skipping!
Property 'summary_embedding' already exists in node '03d8d1'. Skipping!
Property 'summary_embedding' already exists in node 'cc5108'. Skipping!
Property 'summary_embedding' already exists in node 'ed714b'. Skipping!
Property 'summary_embedding' already exists in node 'fd946f'. Sk

Applying [CosineSimilarityBuilder, OverlapScoreBuilder]:   0%|          | 0/2 [00:00<?, ?it/s]

KnowledgeGraph(nodes: 40, relationships: 478)

#### 🎬 Study Note
There is no visial of the kg, but I imagine it would just look like spaghetti anyway with 40 nodes and 478 relationships.

We can save and load our knowledge graphs as follows.

In [11]:
kg.save("loan_data_kg.json")
loan_data_kg = KnowledgeGraph.load("loan_data_kg.json")
loan_data_kg

KnowledgeGraph(nodes: 40, relationships: 478)

Using our knowledge graph, we can construct a "test set generator" - which will allow us to create queries.

In [12]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(llm=generator_llm, embedding_model=embedding_model, knowledge_graph=loan_data_kg)

However, we'd like to be able to define the kinds of queries we're generating - which is made simple by Ragas having pre-created a number of different "QuerySynthesizer"s.

Each of these Synthetsizers is going to tackle a separate kind of query which will be generated from a scenario and a persona.

In essence, Ragas will use an LLM to generate a persona of someone who would interact with the data - and then use a scenario to construct a question from that data and persona.

In [13]:
from ragas.testset.synthesizers import default_query_distribution, SingleHopSpecificQuerySynthesizer, MultiHopAbstractQuerySynthesizer, MultiHopSpecificQuerySynthesizer

query_distribution = [
        (SingleHopSpecificQuerySynthesizer(llm=generator_llm), 0.5),
        (MultiHopAbstractQuerySynthesizer(llm=generator_llm), 0.25),
        (MultiHopSpecificQuerySynthesizer(llm=generator_llm), 0.25),
]

#### ❓ Question #1:

#### ✅ Answer #1
Based on breakout room discussion, plus [Ragas documentation](https://docs.ragas.io/en/stable/concepts/test_data_generation/rag/#multi-hop-query)

Specific vs Abstract:
Specific Query: user queries that require only fact-based retrieval
Abstract Query: user queries that require interpretation or synthesis

Single vs multi-hop:
Single-Hop: user query that only requires retrieving data from one (single) data source / document
Multi-Hop: user query that requires information from two or more sources, and probably reasoning steps to determine the appropriate sources

Finally, we can use our `TestSetGenerator` to generate our testset!

In [14]:
testset = generator.generate(testset_size=10, query_distribution=query_distribution)
testset.to_pandas()

Generating personas:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Scenarios:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/11 [00:00<?, ?it/s]

,user_input,reference_contexts,reference,synthesizer_name
0,What are the defintions of Academic Years in t...,"[Chapter 1 Academic Years, Academic Calendars,...","Chapter 1 Academic Years, Academic Calendars, ...",single_hop_specifc_query_synthesizer
1,What is the regulation that specifies the week...,[Regulatory Citations Academic year minimums: ...,Regulatory Citations Academic year minimums: 3...,single_hop_specifc_query_synthesizer
2,What is the significance of Chapter 3 in rela...,[Inclusion of Clinical Work in a Standard Term...,Inclusion of clinical work in a standard term ...,single_hop_specifc_query_synthesizer
3,What are Non-Term Characteristics in education...,[Non-Term Characteristics A program that measu...,Non-Term Characteristics refer to programs tha...,single_hop_specifc_query_synthesizer
4,How does the TEACH Grrant affect disbursement ...,[both the credit or clock hours and the weeks ...,The TEACH Grant amount a student is eligible t...,single_hop_specifc_query_synthesizer
5,How do standards and definitions of nonstandar...,[<1-hop>\n\nInclusion of Clinical Work in a St...,The context explains that inclusion of clinica...,multi_hop_abstract_query_synthesizer
6,How does the inclusion of clinical work in sta...,[<1-hop>\n\nInclusion of Clinical Work in a St...,The inclusion of clinical work in standard ter...,multi_hop_abstract_query_synthesizer
7,Weeks of instructional time in academic year 3...,"[<1-hop>\n\nChapter 1 Academic Years, Academic...",The context states that for credit-hour progra...,multi_hop_abstract_query_synthesizer
8,Wht Volume 8 shuld I consdier for the clincal ...,[<1-hop>\n\nInclusion of Clinical Work in a St...,"According to the context, Volume 8, Chapter 3 ...",multi_hop_specific_query_synthesizer
9,H0w do Appendix A and Appendix B relate to the...,[<1-hop>\n\nboth the credit or clock hours and...,Appendix A provides guidance on calculating Pe...,multi_hop_specific_query_synthesizer


### Abstracted SDG

The above method is the full process - but we can shortcut that using the provided abstractions!

This will generate our knowledge graph under the hood, and will - from there - generate our personas and scenarios to construct our queries.



In [15]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(llm=generator_llm, embedding_model=generator_embeddings)
dataset = generator.generate_with_langchain_docs(docs[:20], testset_size=10)

Applying HeadlinesExtractor:   0%|          | 0/17 [00:00<?, ?it/s]

Applying HeadlineSplitter:   0%|          | 0/20 [00:00<?, ?it/s]

unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node


Applying SummaryExtractor:   0%|          | 0/31 [00:00<?, ?it/s]

Property 'summary' already exists in node '6e5813'. Skipping!
Property 'summary' already exists in node 'c14b49'. Skipping!
Property 'summary' already exists in node '6c9ea3'. Skipping!
Property 'summary' already exists in node 'f82ced'. Skipping!
Property 'summary' already exists in node '626070'. Skipping!
Property 'summary' already exists in node 'bdbdce'. Skipping!
Property 'summary' already exists in node 'ec1e61'. Skipping!
Property 'summary' already exists in node '670430'. Skipping!
Property 'summary' already exists in node 'b1ae60'. Skipping!
Property 'summary' already exists in node '46fbb0'. Skipping!
Property 'summary' already exists in node '2b75de'. Skipping!
Property 'summary' already exists in node '42cd93'. Skipping!
Property 'summary' already exists in node '6ca563'. Skipping!
Property 'summary' already exists in node '21770e'. Skipping!


...

Property 'summary_embedding' already exists in node '6e5813'. Skipping!
Property 'summary_embedding' already exists in node 'f82ced'. Skipping!
Property 'summary_embedding' already exists in node '21770e'. Skipping!
Property 'summary_embedding' already exists in node 'c14b49'. Skipping!
Property 'summary_embedding' already exists in node '6ca563'. Skipping!
Property 'summary_embedding' already exists in node 'ec1e61'. Skipping!
Property 'summary_embedding' already exists in node '6c9ea3'. Skipping!
Property 'summary_embedding' already exists in node '42cd93'. Skipping!
Property 'summary_embedding' already exists in node 'b1ae60'. Skipping!
Property 'summary_embedding' already exists in node 'bdbdce'. Skipping!
Property 'summary_embedding' already exists in node '670430'. Skipping!
Property 'summary_embedding' already exists in node '626070'. Skipping!
Property 'summary_embedding' already exists in node '46fbb0'. Skipping!
Property 'summary_embedding' already exists in node '2b75de'. Sk

Applying [CosineSimilarityBuilder, OverlapScoreBuilder]:   0%|          | 0/2 [00:00<?, ?it/s]

Generating personas:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Scenarios:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/12 [00:00<?, ?it/s]

In [16]:
dataset.to_pandas()

,user_input,reference_contexts,reference,synthesizer_name
0,What is the significance of Volume 2 in the c...,"[Chapter 1 Academic Years, Academic Calendars,...",The provided context does not specify the sign...,single_hop_specifc_query_synthesizer
1,What does 34 CFR 668.3(b) specify regarding we...,[Regulatory Citations Academic year minimums: ...,34 CFR 668.3(b) pertains to weeks of instructi...,single_hop_specifc_query_synthesizer
2,What is the significance of Chapter 3 in relat...,[Inclusion of Clinical Work in a Standard Term...,Inclusion of clinical work in a standard term ...,single_hop_specifc_query_synthesizer
3,Is Federal Work-Study a payment period program?,[Non-Term Characteristics A program that measu...,"No, the Federal Work-Study (FWS) Program is an...",single_hop_specifc_query_synthesizer
4,how differences between standard and nonstanda...,[<1-hop>\n\nInclusion of Clinical Work in a St...,The context explains that clinical work outsid...,multi_hop_abstract_query_synthesizer
5,Based on the information about academic years ...,"[<1-hop>\n\nChapter 1 Academic Years, Academic...",A school determines the appropriate payment pe...,multi_hop_abstract_query_synthesizer
6,Wht are the diffrent acedemic yers for varous ...,"[<1-hop>\n\nChapter 1 Academic Years, Academic...",The context explains that each eligible progra...,multi_hop_abstract_query_synthesizer
7,How does the disbursement of funds based on pa...,[<1-hop>\n\nInclusion of Clinical Work in a St...,The disbursement of funds based on payment per...,multi_hop_abstract_query_synthesizer
8,How do the disbursement timing rules outlined ...,[<1-hop>\n\nDisbursement Timing in Subscriptio...,The disbursement timing rules in Appendix B pr...,multi_hop_specific_query_synthesizer
9,Chapter 2 disbursement rules need to know how ...,[<1-hop>\n\nDisbursement Timing in Subscriptio...,Chapter 2 explains disbursement timing and mul...,multi_hop_specific_query_synthesizer


We'll need to provide our LangSmith API key, and set tracing to "true".

# 🤝 BREAKOUT ROOM #2

## Task 4: LangSmith Dataset

Now we can move on to creating a dataset for LangSmith!

First, we'll need to create a dataset on LangSmith using the `Client`!

We'll name our Dataset to make it easy to work with later.

In [18]:
from langsmith import Client

client = Client()

#dataset_name = "Loan Synthetic Data"
dataset_name = f"Loan Synthetic Data - {uuid4().hex[:8]}"


langsmith_dataset = client.create_dataset(
    dataset_name=dataset_name,
    description="Loan Synthetic Data"
)
print(f"Dataset name: {dataset_name}")


Dataset name: Loan Synthetic Data - 78f96140


We'll iterate through the RAGAS created dataframe - and add each example to our created dataset!

> NOTE: We need to conform the outputs to the expected format - which in this case is: `question` and `answer`.

In [19]:
for data_row in dataset.to_pandas().iterrows():
  client.create_example(
      inputs={
          "question": data_row[1]["user_input"]
      },
      outputs={
          "answer": data_row[1]["reference"]
      },
      metadata={
          "context": data_row[1]["reference_contexts"]
      },
      dataset_id=langsmith_dataset.id
  )

## Basic RAG Chain

Time for some RAG!


In [20]:
rag_documents = docs

To keep things simple, we'll just use LangChain's recursive character text splitter!


#### 🎬 Study Notes
Overlap is discussed more in Session 8

In [21]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap = 50
)

rag_documents = text_splitter.split_documents(rag_documents)

We'll create our vectorstore using OpenAI's [`text-embedding-3-small`](https://platform.openai.com/docs/guides/embeddings/embedding-models) embedding model.

In [22]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

As usual, we will power our RAG application with Qdrant!

In [23]:
from langchain_community.vectorstores import Qdrant

vectorstore = Qdrant.from_documents(
    documents=rag_documents,
    embedding=embeddings,
    location=":memory:",
    collection_name="Loan RAG"
)

In [24]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 10})

To get the "A" in RAG, we'll provide a prompt.

In [25]:
from langchain.prompts import ChatPromptTemplate

RAG_PROMPT = """\
Given a provided context and question, you must answer the question based only on context.

If you cannot answer the question based on the context - you must say "I don't know".

Context: {context}
Question: {question}
"""

rag_prompt = ChatPromptTemplate.from_template(RAG_PROMPT)

For our LLM, we will be using TogetherAI's endpoints as well!

We're going to be using Meta Llama 3.1 70B Instruct Turbo - a powerful model which should get us powerful results!

#### 🎬 Study Note
Above md cell appears to be incorrect; perhaps leftover from older cohort??


In [26]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4.1-mini")

Finally, we can set-up our RAG LCEL chain!

In [27]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain.schema import StrOutputParser

rag_chain = (
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    | rag_prompt | llm | StrOutputParser()
)

In [28]:
rag_chain.invoke({"question" : "What kinds of loans are available?"})

'The kinds of loans available mentioned in the context are:\n\n- Direct Subsidized Loan  \n- Direct Unsubsidized Loan  \n- Direct PLUS Loan (or student Federal PLUS Loan)  \n- Federal Stafford Loans (Subsidized and Unsubsidized)  \n- Federal SLS Loans  \n- Federal PLUS Loans  \n\nNote that Subsidized and Unsubsidized Federal Stafford Loans, Federal SLS Loans, and Federal PLUS Loans were made under the Federal Family Education Loan (FFEL) Program, which ended effective July 1, 2010, so no new loans are made under FFEL.  \n\nDirect Subsidized, Direct Unsubsidized, and Direct PLUS Loans are the current loans referenced. Additionally, Direct Unsubsidized Loans and Direct PLUS Loans can be used to replace the Satisfactory Academic Progress Indicator (SAI) and cover unmet need, and Direct Subsidized Loans are available only to undergraduate students. Graduate or professional students are eligible only for Direct Unsubsidized Loans and Direct PLUS Loans, not Direct Subsidized Loans.'

## LangSmith Evaluation Set-up

We'll use OpenAI's GPT-4.1 as our evaluation LLM for our base Evaluators.

In [29]:
eval_llm = ChatOpenAI(model="gpt-4.1")

#### 🎬 Study Note

using more capable model for evals (so that the "grader knows more than the student")

We'll be using a number of evaluators - from LangSmith provided evaluators, to a few custom evaluators!

In [30]:
from langsmith.evaluation import LangChainStringEvaluator, evaluate

qa_evaluator = LangChainStringEvaluator("qa", config={"llm" : eval_llm})

labeled_helpfulness_evaluator = LangChainStringEvaluator(
    "labeled_criteria",
    config={
        "criteria": {
            "helpfulness": (
                "Is this submission helpful to the user,"
                " taking into account the correct reference answer?"
            )
        },
        "llm" : eval_llm
    },
    prepare_data=lambda run, example: {
        "prediction": run.outputs["output"],
        "reference": example.outputs["answer"],
        "input": example.inputs["question"],
    }
)

empathy_evaluator = LangChainStringEvaluator(
    "criteria",
    config={
        "criteria": {
            "empathy": "Is this response empathetic? Does it make the user feel like they are being heard?",
        },
        "llm" : eval_llm
    }
)

#### 🏗️ Activity #2:

Highlight what each evaluator is evaluating.

- `qa_evaluator`:
- `labeled_helpfulness_evaluator`:
- `empathy_evaluator`:

#### ✅ Response to Activity #2
Please see detailed response in attached PDF here: [Activity2_response_evaluator_explanation.pdf](./Activity2_response_evaluator_explanation.pdf) 

## LangSmith Evaluation

#### 🎬 Study Note
using this md cell as a handy place to preserve the langsmith URLs

Initial run:
View the evaluation results for experiment: 'earnest-meat-17' at:

https://smith.langchain.com/o/6a26aaa0-1b14-4cdb-933f-5cb4da328a84/datasets/f76e0c37-88b6-4b5b-aa85-328dd0d8cc9e/compare?selectedSessions=25695b40-df30-4be2-9abc-089ccc6a1e13

Second run:

https://smith.langchain.com/o/6a26aaa0-1b14-4cdb-933f-5cb4da328a84/datasets/c54ceb9f-315d-48c2-8030-ac40694805c8/compare?selectedSessions=b5140bba-9cc0-4184-92e6-d1d6e2c78499



In [41]:
evaluate(
    rag_chain.invoke,
    data=dataset_name,
    evaluators=[
        qa_evaluator,
        labeled_helpfulness_evaluator,
        empathy_evaluator
    ],
    metadata={"revision_id": "default_chain_init"},
)

View the evaluation results for experiment: 'timely-suggestion-82' at:
https://smith.langchain.com/o/6a26aaa0-1b14-4cdb-933f-5cb4da328a84/datasets/c54ceb9f-315d-48c2-8030-ac40694805c8/compare?selectedSessions=b0ec745f-4fdf-4a74-916c-a116be27c7a8




0it [00:00, ?it/s]

,inputs.question,outputs.output,error,reference.answer,feedback.correctness,feedback.helpfulness,feedback.empathy,execution_time,example_id,id
0,How do Volume 2 and Volume 7 relate to academi...,I don't know.,None,Volume 2 discusses the academic year requireme...,0,0,0,1.242778,69b5663b-f524-41b8-b91e-91b7b5cf7323,d30b7ca8-32f3-4279-bd7e-71c3716e7c67
1,which volume 2 and volume 8 are relted to disb...,"Based on the provided context, Volume 3, Chapt...",None,"Volume 2 discusses academic years, instruction...",0,0,0,7.381465,73520e07-ca17-4573-a4d5-b09fb51eb2e6,36701ab6-da24-469c-8f08-e794d363c687
2,Chapter 2 disbursement rules need to know how ...,I don't know.,None,Chapter 2 explains disbursement timing and mul...,0,0,0,0.826293,2c5da648-6353-49be-9856-33489447ef6c,c6e51230-991f-441d-90c8-cb427b5b479b
3,How do the disbursement timing rules outlined ...,The disbursement timing rules outlined in Appe...,None,The disbursement timing rules in Appendix B pr...,1,1,0,10.621148,317e50eb-da1c-419c-bf3e-41cb35f97cb9,9d995018-e436-495f-a6a3-4449fc5d748f
4,How does the disbursement of funds based on pa...,I don't know.,None,The disbursement of funds based on payment per...,0,0,0,0.876769,aee122d8-ea00-4f09-900f-23451161c3bd,13ccf8ec-c323-4f2b-9a1d-073fd91a70e5
5,Wht are the diffrent acedemic yers for varous ...,The context explains that every eligible progr...,None,The context explains that each eligible progra...,0,0,0,4.702053,6ef62768-e2cc-412d-8d79-03a41d198da8,193f8f08-5551-4f5a-9d07-01b977171a2d
6,Based on the information about academic years ...,"Based on the provided context, a school determ...",None,A school determines the appropriate payment pe...,1,1,0,8.875960,542a83e0-10e2-432f-b1a2-9e5560d3c512,018843cb-5dd8-4f8f-b16f-d71719ec0981
7,how differences between standard and nonstanda...,Based on the provided context:\n\n**Difference...,None,The context explains that clinical work outsid...,1,1,0,7.894249,1f2ce2fa-09b8-422d-9213-28c3286f96c1,d8adcad1-63c9-49c3-9544-3875c3fae88a
8,Is Federal Work-Study a payment period program?,"No, Federal Work-Study (FWS) is not a payment ...",None,"No, the Federal Work-Study (FWS) Program is an...",1,1,0,1.339127,f7c9b49d-d4a2-471a-be48-feec57e4da1e,3be3d525-c857-438f-8af8-b93da90b4f05
9,What is the significance of Chapter 3 in relat...,Chapter 3 provides additional guidance on cert...,None,Inclusion of clinical work in a standard term ...,1,0,0,1.399628,65a2454a-115f-4a55-a43c-61167df3206a,bd8a0311-8a39-4e88-aebe-66ca7919a126


## Dope-ifying Our Application - Empifying!

We'll be making a few changes to our RAG chain to increase its performance on our SDG evaluation test dataset!

- Include a "dope" prompt augmentation
- Use larger chunks
- Improve the retriever model to: `text-embedding-3-large`

Let's see how this changes our evaluation!

#### 🎬 Study Note: 
I think we're "empifying" not dope-ifying 😃

In [32]:
EMPATHY_RAG_PROMPT = """\
Given a provided context and question, you must answer the question based only on context.

If you cannot answer the question based on the context - you must say "I don't know".

You must answer the question using empathy and kindness, and make sure the user feels heard.

Context: {context}
Question: {question}
"""

empathy_rag_prompt = ChatPromptTemplate.from_template(EMPATHY_RAG_PROMPT)

In [33]:
rag_documents = docs

In [34]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 50
)

rag_documents = text_splitter.split_documents(rag_documents)

#### ❓Question #2:

Why would modifying our chunk size modify the performance of our application?

#### ✅ Answer #2
There are tradeoffs.
Larger chunks can capture more complete ideas. This reduces the chance that important chunks will fail to be retrieved because they are too fragmented for the retriever to find the relevance. 

However, larger chunks can also increase noisiness, and LLMs can get distracted or lose focus if there is too much context. (Increased coverage; potentially decreased coherence.)

Larger chunks also cost (and potentially waste) more token usage.



In [35]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

#### ❓Question #3:

Why would modifying our embedding model modify the performance of our application?

#### ✅ Answer #3:
The "text-embedding-3-large" embedding model uses 3,072 dimensional vectors (twice as many as the 'small' one we've used before).
More dimensions means capturing more aspects of the data, creating better relationships between embeddings, leading to better semantic matching, and making retrieval more accurate.

A model with higher dimensionality is also better able to handle larger chunks, without overly compressing semantic meaning. Larger chunks allow for more robust context, as noted in Answer #2 above.

In short, better embedding should yield better retrieval quality. 

#### 🎬 Study Note:

You can also improve embedding models by  custom-training or fine-tuning them. 
This is more feasible than fine tuning an LLM (more bang for your buck).

In breakout, Can shared that Google has good embedding models, but more expensive than OpenAI.
Cohere makes good inexpensive models, and also good at re-ranking models (see Session 8). 

In [36]:
vectorstore = Qdrant.from_documents(
    documents=rag_documents,
    embedding=embeddings,
    location=":memory:",
    collection_name="Loan Data for RAG"
)

In [37]:
retriever = vectorstore.as_retriever()

Setting up our new and improved DOPE RAG CHAIN.

In [38]:
empathy_rag_chain = (
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    | empathy_rag_prompt | llm | StrOutputParser()
)

Let's test it on the same output that we saw before.

In [39]:
empathy_rag_chain.invoke({"question" : "What kinds of loans are available?"})

"Thank you for your question—it's important to understand the loan options available to help manage educational costs. Based on the information provided, there are several types of federal student loans you may consider:\n\n1. **Direct Subsidized Loans:** These are loans where the government pays the interest while you're in school at least half-time. They are limited to the student's financial need (Cost of Attendance minus financial aid and other aid).\n\n2. **Direct Unsubsidized Loans:** These loans are not based on financial need, and you are responsible for interest that accrues during all periods.\n\n3. **Direct PLUS Loans:** These are available to parents of dependent students (assuming eligibility) to cover the student's cost of attendance not covered by other aid. There is no fixed loan limit, but the amount cannot exceed the cost of attendance minus other financial aid.\n\nAdditionally, if a dependent student's parent is unable to get a Direct PLUS Loan, the student may be el

Finally, we can evaluate the new chain on the same test set!

#### 🎬 Study Note
Using this cell as a handy place to preserve the eval URLs

Empified eval run #1:
View the evaluation results for experiment: 'impressionable-distribution-50' at:

https://smith.langchain.com/o/6a26aaa0-1b14-4cdb-933f-5cb4da328a84/datasets/f76e0c37-88b6-4b5b-aa85-328dd0d8cc9e/compare?selectedSessions=8c48bb0a-c2d5-440e-a963-21e3c2629b22


Empified eval run #2:

View the evaluation results for experiment: 'damp-parcel-47' at:

https://smith.langchain.com/o/6a26aaa0-1b14-4cdb-933f-5cb4da328a84/datasets/c54ceb9f-315d-48c2-8030-ac40694805c8/compare?selectedSessions=7f1097ea-de6b-4a1c-9aed-20e73c8e8e31

In [40]:
evaluate(
    empathy_rag_chain.invoke,
    data=dataset_name,
    evaluators=[
        qa_evaluator,
        labeled_helpfulness_evaluator,
        empathy_evaluator
    ],
    metadata={"revision_id": "empathy_rag_chain"},
)

View the evaluation results for experiment: 'damp-parcel-47' at:
https://smith.langchain.com/o/6a26aaa0-1b14-4cdb-933f-5cb4da328a84/datasets/c54ceb9f-315d-48c2-8030-ac40694805c8/compare?selectedSessions=7f1097ea-de6b-4a1c-9aed-20e73c8e8e31




0it [00:00, ?it/s]

,inputs.question,outputs.output,error,reference.answer,feedback.correctness,feedback.helpfulness,feedback.empathy,execution_time,example_id,id
0,How do Volume 2 and Volume 7 relate to academi...,Thank you for your thoughtful question. Based ...,None,Volume 2 discusses the academic year requireme...,0,0,1,5.090199,69b5663b-f524-41b8-b91e-91b7b5cf7323,ff3b8373-f290-4d28-8651-494e8ca05b2d
1,which volume 2 and volume 8 are relted to disb...,Thank you for your thoughtful question. Based ...,None,"Volume 2 discusses academic years, instruction...",0,0,1,3.691141,73520e07-ca17-4573-a4d5-b09fb51eb2e6,876db0b0-a915-4a46-ad10-f5c9373bffab
2,Chapter 2 disbursement rules need to know how ...,Thank you for your thoughtful question. It’s c...,None,Chapter 2 explains disbursement timing and mul...,0,0,1,3.953958,2c5da648-6353-49be-9856-33489447ef6c,17f7059b-fd76-4b68-aebd-3a37d3cec5c2
3,How do the disbursement timing rules outlined ...,Thank you for your thoughtful question. From t...,None,The disbursement timing rules in Appendix B pr...,1,1,1,8.035077,317e50eb-da1c-419c-bf3e-41cb35f97cb9,497f3002-65fa-43c5-bcc4-ac43c7896104
4,How does the disbursement of funds based on pa...,Thank you for your thoughtful question. I unde...,None,The disbursement of funds based on payment per...,1,1,1,5.683577,aee122d8-ea00-4f09-900f-23451161c3bd,c496af54-cd2e-412b-9534-38b4c2ada6dd
5,Wht are the diffrent acedemic yers for varous ...,Thank you for your question! I understand you'...,None,The context explains that each eligible progra...,1,0,1,8.170590,6ef62768-e2cc-412d-8d79-03a41d198da8,d3b9e23a-1adb-452a-a4ae-f3ec105bb005
6,Based on the information about academic years ...,Thank you for your thoughtful question. I unde...,None,A school determines the appropriate payment pe...,1,1,1,13.221215,542a83e0-10e2-432f-b1a2-9e5560d3c512,75a36479-b8be-4087-8902-17f0f231da89
7,how differences between standard and nonstanda...,Thank you for your thoughtful question. It’s c...,None,The context explains that clinical work outsid...,1,1,1,9.866264,1f2ce2fa-09b8-422d-9213-28c3286f96c1,f247abad-becc-4f75-9c4c-63b86a92f259
8,Is Federal Work-Study a payment period program?,Thank you for your question. Based on the cont...,None,"No, the Federal Work-Study (FWS) Program is an...",1,1,1,5.049852,f7c9b49d-d4a2-471a-be48-feec57e4da1e,9fe203a8-9f64-4f62-8ca6-ac4fc3c8cf2c
9,What is the significance of Chapter 3 in relat...,Thank you for your thoughtful question. Based ...,None,Inclusion of clinical work in a standard term ...,1,0,1,4.142948,65a2454a-115f-4a55-a43c-61167df3206a,634180c9-98e9-4640-802f-ebc38c6d31b2


#### 🏗️ Activity #3:

Provide a screenshot of the difference between the two chains, and explain why you believe certain metrics changed in certain ways.

#### ✅ Response to Activity #3
The empathy_evaluator improved from 0% during the baseline (rag_chain) to 100% in the empathy_rag_chain evaluation.
The correctness (based on QA evaluator) improved improved from 7 to 9 (out of 12).
Helpfulness also rose slightly from 5 to 6 (out of 12).

As shown in [this screenshot.](./Screenshot_chain_comparison2.png)

These results match my expectations, based on the three changes that were made:
>- Including an "empathy" prompt augmentation
>- Increaseing chunk sise from 500 to 1,000
>- Changing the retriever model from `text-embedding-3-small` to: `text-embedding-3-large`

**Empathy** (empathy_evaluator): 
The original chain had no instructions about empathy, whereas the “empified” chain was explicitly told to: 
>“answer the question using empathy and kindness, and make sure the user feels heard.”

**Correctness** (qa_evaluator):
QA accuracy depends on relevance and completeness of the retrieved context. 
Larger chunks mean more complete context (as noted in Answer #2), and the better embedding model could improve semantic matching for retrieval (as noted in Answer #3)

**Helpfulness** (labeled_helpfulness_evaluator):
This is a more subjective measure, and the slight increase may have been a second-order effect of the improved empathy and accuracy.



